In [7]:
import logging
import pymongo
import cobrakbase
logger = logging.getLogger(__name__)

In [2]:
kbase = cobrakbase.KBaseAPI()

In [3]:
modelseed_local = cobrakbase.modelseed.from_local('../../../../ModelSEEDDatabase')

In [5]:
%run ../../../annotation-server/annotation_api_neo4j.py
host, port, user, pwd = ("0.0.0.0", 7687, "neo4j", "123585")
def init_annotation_api(host, port, user, pwd):
    annotation_api = AnnotationApiNeo4j(user=user, pwd=pwd, port=port, host=host)
    annotation_api.neo4j_graph = Graph("http://neo4j:" + pwd + "@" + host + ":7474")
    annotation_api.matcher = NodeMatcher(annotation_api.neo4j_graph)
    annotation_api.r_matcher = RelationshipMatcher(annotation_api.neo4j_graph)
    annotation_api.init_constraints()
    return annotation_api

annotation_api = init_annotation_api(host, port, user, pwd)

In [8]:
aclient = pymongo.MongoClient('mongodb://192.168.1.15:27017/')
database = aclient['annotation']

In [144]:
template_o = kbase.get_from_ws('GramNegModelTemplateV2', 'NewKBaseModelTemplates')

In [143]:
%run ../../export_template_to_kbase.py

In [145]:
output = export_template(template_o, modelseed_local, annotation_api, database, 
                         'fungi', clear_roles=True, clear_reactions=True, reaction_list=['rxn00001_c'])

2119
updated 1042
removed 1219
undeclared compounds 0
undeclared roles 0
undeclared complexes 0
dict_keys(['fungi', 'template_v3', 'test'])
6373


0


In [146]:
result = kbase.save_object('template_reaction_list', 'filipeliu:narrative_1582914694010', 'KBaseFBA.NewModelTemplate', output.get_data())

In [114]:
template_o = kbase.get_from_ws('GramNegModelTemplateV2', 'NewKBaseModelTemplates')

In [115]:
clear_reactions = True
reaction_list = ['rxn00001_c']
clear_roles = True
clear_complexes = False
mongo_database = database
modelseed = modelseed_local
annotation_namespace = 'fungi'

In [116]:
data_copy = copy.deepcopy(template_o.get_data())
template = NewModelTemplate(data_copy, template_o.info, None, 'tftr', 'tcpx')

In [117]:
tm = TemplateManipulator(template, None)
template_reactions_filter = tm.clean_template('ModelSEED')
print(len(template_reactions_filter))
updated, removed = tm.clear_orphan_roles()
print('updated', len(updated))
print('removed', len(removed))

validator = NewModelTemplateValidator(template)
validator.validate_compounds()
validator.validate()
print('undeclared compounds', len(validator.undec_compounds))
print('undeclared roles', len(validator.undec_roles))
print('undeclared complexes', len(validator.undec_complexes))

if clear_reactions or reaction_list is not None:
    template.reactions.clear()
    template.reactions._dict.clear()
if clear_roles:
    template.complexes.clear()
    template.complexes._dict.clear()
    template.roles.clear()
    template.roles._dict.clear()
    template.role_set_to_cpx.clear()
    template.search_name_to_role_id.clear()
    template.role_last_id = 0
    template.complex_last_id = 0
if clear_complexes:
    template.complexes.clear()
    template.complexes._dict.clear()
    template.role_set_to_cpx.clear()
    template.complex_last_id = 0
# if roles or complexes were removed clear from reactions
if clear_roles or clear_complexes:
    for trxn in template_o.reactions:
        trxn.templatecomplex_refs.clear()

2119
updated 1042
removed 1219
undeclared compounds 0
undeclared roles 0
undeclared complexes 0


In [118]:
tc = TemplateCuration(template, mongo_database, annotation_api)
tm = TemplateManipulator(template, modelseed)
a = tc.get_reaction_annotation()
print(a.keys())

dict_keys(['fungi', 'template_v3', 'test'])


In [124]:
search_name_to_role_id = tm.get_search_name_to_role_id()
for k in search_name_to_role_id:
    if len(search_name_to_role_id[k]) > 1:
        print(k)
accept, remove = tc.get_curation_data(annotation_namespace)
if reaction_list is not None:
    accept = dict(filter(lambda x: x[0] in reaction_list, accept.items()))
    remove = dict(filter(lambda x: x[0] in reaction_list, remove.items()))
test_accept = dict(filter(lambda x: len(x[1]) > 0, accept.items()))
test_remove = dict(filter(lambda x: len(x[1]) > 0, remove.items()))

In [126]:
test_remove

{'rxn00001_c': {'352447', '416681', '5544039'}}

In [127]:
test_accept_sn_to_roles = tc.get_roles_to_add(test_accept, search_name_to_role_id)
for role_sn in test_accept_sn_to_roles:
    role_name = list(test_accept_sn_to_roles[role_sn])[0]
    template.add_role(role_name)
search_name_to_role_id = tm.get_search_name_to_role_id()

In [128]:
print(template.reactions)

[]


In [129]:
# remove all reactions exclude from template
remove_reactions = tc.get_disabled_reactions(annotation_namespace)

In [130]:
reactions_in_template = set(map(lambda x: x.id, template.reactions))

# strip complexes from reactions in remove set
for trxn_id in remove_reactions:
    if trxn_id in reactions_in_template:
        trxn = template.get_reaction(trxn_id)
        trxn.templatecomplex_refs.clear()

In [131]:
# add new reactions
reactions_to_add = []
for doc in tc.curation_api['templates_reactions'].find():
    template_rxn_id, template_id = doc['_id'].split('@')
    if template_id == annotation_namespace and template_rxn_id not in remove_reactions and \
            (reaction_list is None or template_rxn_id in reaction_list):
        if 'cmp' in doc:
            if template_rxn_id not in reactions_in_template and \
                    'annotation' in doc and \
                    'seed__DOT__reaction' in doc['annotation']:
                seed_id = doc['annotation']['seed__DOT__reaction']
                trxn_b = tm.build_template_reaction_from_modelseed(seed_id, doc['cmp'])
                reactions_to_add.append(NewModelTemplateReaction(trxn_b))

In [82]:
oo = set()
for o in reactions_to_add:
    if o.id == 'rxn11811_c':
        print(o.id)
    if o.id not in oo:
        oo.add(o.id)
    else:
        print(o.id)

rxn11811_c


In [132]:
template.reactions += reactions_to_add
reactions_in_template = set(map(lambda x: x.id, template.reactions))

In [133]:
for trxn_id in set(test_remove):
    if trxn_id in reactions_in_template and trxn_id not in remove_reactions:
        template_rxn = template.get_reaction(trxn_id)
        role_change = tc.get_role_change(trxn_id, {}, test_remove)
        # role_change = get_role_change2(tc, rxn_id, {}, test_remove)
        # print(trxn.id)
        nfunction = tc.update_roles(template_rxn, role_change, search_name_to_role_id, True)
    else:
        logger.warning('%s', trxn_id)


def refresh(template):
    template.role_set_to_cpx = {}
    template.search_name_to_role_id = {}
    for role in template.data['roles']:
        template.search_name_to_role_id[normalize_role(role['name'])] = role['id']
    for cpx in template.data['complexes']:
        roles = set()
        for complexrole in cpx['complexroles']:
            role_id = complexrole['templaterole_ref'].split('/')[-1]
            roles.add(role_id)
        # print(cpx, roles)
        template.role_set_to_cpx[';'.join(sorted(roles))] = cpx['id']

refresh(template)

In [134]:
for trxn_id in set(test_accept):
    if trxn_id in reactions_in_template and trxn_id not in remove_reactions  and \
            (reaction_list is None or trxn_id in reaction_list):
        template_rxn = template.get_reaction(trxn_id)
        try:
            role_change = tc.get_role_change(trxn_id, test_accept, {})
            # role_change = get_role_change2(tc, rxn_id, test_accept, {})
            nfunction = tc.update_roles(template_rxn, role_change, search_name_to_role_id, True)
        except Exception as e:
            print(template_rxn.id, e)
    else:
        logger.warning('%s', trxn_id)

In [142]:
reaction_annotation = tc.get_reaction_annotation()
role_uids = set()
system_accept_role_uids = set()
for rxn_id in reaction_annotation[annotation_namespace]:
    if reaction_list is None or rxn_id in reaction_list:
        for role_id in reaction_annotation[annotation_namespace][rxn_id]['user']:
            score = reaction_annotation[annotation_namespace][rxn_id]['current'][str(role_id)]
            if score == 'opt_score1':
                user_log = reaction_annotation[annotation_namespace][rxn_id]['user'][role_id]
                if len(user_log) > 1 or 'system' not in user_log:
                    role_uids.add(role_id)
                    # print(rxn_id, role_id, user_log, score)
                else:
                    system_accept_role_uids.add(role_id)

print(len(role_uids))
for role_uid in role_uids:
    a, role_ids = tc.get_function(role_uid, search_name_to_role_id)
    for role_id in role_ids:
        role = template.get_role(role_id)
        if role:
            # print(role_uid, role['id'], role['source'])
            role['source'] = 'ModelSEED'

rxn00001_c
0


In [61]:
result = kbase.save_object('template_clear_roles', 'filipeliu:narrative_1582914694010', 'KBaseFBA.NewModelTemplate', template.get_data())

In [147]:
for rxn_id in reaction_annotation[annotation_namespace]:
    print(rxn_id)
    break

rxn10121


In [148]:
%run ../../curation_api.py

In [149]:
curation_api = CurationApi(aclient)

In [150]:
rxn_annotation_manual_function = curation_api.get_manual_function('rxn15046_c', 'fungi')

In [151]:
rxn_annotation_manual_function

{'_id': 'rxn15046_c@fungi',
 'functions': {'502156': True},
 'log': [{'timestamp': 1601536929,
   'user_id': 'fliu',
   'action': True,
   'target': '502156'}]}

In [153]:
f = annotation_api.get_function_by_uid(502156)

In [154]:
f.value

'"DNA-binding protein, CopG family"'